# Working with Elasticsearch

### For guidance, see [Elastic Stack and Product Documentation](https://www.elastic.co/guide/index.html)

### Import necessary modules

In [1]:
import json
import pathlib
import requests
import time
# twittertools is my local twittertools.py module
import twittertools

### Define Elasticsearch server URI

In [2]:
elasticsearch_server = 'http://localhost:9200'

### Print current indices

In [3]:
r = requests.get(elasticsearch_server + '/_cat/indices')
print(r.text)

yellow open .kibana Yuy-vovlQyeUPVim2PcdFw 1 1    2 0 10.9kb 10.9kb
yellow open twitter hAyoLMagQia7L-gNb6WhWQ 5 1 3242 0    2mb    2mb



### Create Authenticated TwitterTools object

In [4]:
filepath = pathlib.Path.home().joinpath('.twitter', 'credentials.json')
twt = twittertools.TwitterTools(filepath)

### Get tweets from @pourmecoffee's timeline

In [5]:
screen_name = 'pourmecoffee'
tweets = twt.get_user_timeline(screen_name)
print(f"{len(tweets)} tweets retrieved from @{screen_name}'s timeline")

3242 tweets retrieved from @pourmecoffee's timeline


### Index all tweets

In [6]:
# For this demonstration only, delete any existing /twitter index
tweet_index_uri = elasticsearch_server + '/twitter'
requests.delete(tweet_index_uri)

# Index the tweets
post_uri = tweet_index_uri + '/tweets'
total = 0
for tweet in tweets:
    source = twittertools.unpack_tweet(tweet)
    r = requests.post(post_uri, json=source)
    if r.status_code == 201:
        total += 1
print(f'{total} tweets indexed')

3242 tweets indexed


### Confirm number of tweets indexed

In [7]:
sleep_time = 0.50
search_uri = post_uri + '/_search'
while True:
    results = requests.get(search_uri).json()
    if results['hits']['total'] == total:
        break
    time.sleep(sleep_time)
print(f'Found {total} tweets')

Found 3242 tweets
